# Огавление
1. [Описание проекта](#description)
2. [Открытие данных](#1)
3. [Предобработка данных](#2)
    * [Обработка пропусков](#3)
    * [Замена типа данных](#4)
    * [Обработка дубликатов](#5)  
    * [Лемматизация](#6)
    * [Категоризация данных](#7)
    
    
4. [Ответы на вопросы](#8) 
5. [Общий вывод](#9) 

## Описание проекта  <a id="description"></a>
### Исследование надёжности заёмщиков 

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию.   <a id="1"></a>

In [22]:
import pandas as pd

data = pd.read_csv('/datasets/data.csv')

data.to_csv('dlya_analiza.csv')

In [23]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Вывод

- Всего в таблице 12 столбцов.
- Количество значений в столбцах различается. 
- Часть данных отсутствует в столбцах total_income и days_employed, займемся их предобработкой далее

### Шаг 2. Предобработка данных <a id="2"></a>

In [24]:
data['days_employed'].value_counts()

-986.927316     1
-7026.359174    1
-4236.274243    1
-6620.396473    1
-1238.560080    1
               ..
-2849.351119    1
-5619.328204    1
-448.829898     1
-1687.038672    1
-582.538413     1
Name: days_employed, Length: 19351, dtype: int64

In [25]:
# Так как при просмотре таблицы в столбце days_employed были обнаружены отрицательные значения, что могло быть
# вызванно технической ошибкой, возьмем эти же значения по модулю

data['days_employed'] = abs(data['days_employed'])

In [26]:
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


### Обработка пропусков <a id="3"></a>

In [27]:



#print(data.head())
#print(data[data['days_employed'].isnull()].tail(10))
#print(data[data['total_income'].isnull()].count())

# При просмотре таблицы, в столбцах 'days_employed' и 'total_income' были обнаруженны пустые значения, т.к. значения числовые
# было принято решение заполнить пропуски средним значением по столбцам

data['days_employed'] = data['days_employed'].fillna(data['days_employed'].mean())
data['total_income'] = data['total_income'].fillna(data['total_income'].mean())

In [28]:
data.info() 
# Больше нулевых значений в данных нет.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [29]:
# При просмотре столбца 'children', методом '.value_counts()' были выявлены аномальные значения (-1, 20)
# Скорее всего это вызвано ошибкой человека вводившего данные
# Решение было простым, заменить -1 на 1, а 20 на 2

data.loc[data['children'] == -1, 'children'] = 1
data.loc[data['children'] == 20, 'children'] = 2

# При просмотре столбца 'dob_years', методом '.value_counts()' было выявлено аномальное значение (0)
# Скорее всего это техническая ошибка, было принято решение заменить 0 на среднее значение по столбцу

data['dob_years'].value_counts()
mean_dob_years = data['dob_years'].mean()
data.loc[data['dob_years'] == 0, 'dob_years'] = mean_dob_years

#print(mean_dob_years)
#print(data.head())

print(data.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null float64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(3), int64(4), object(5)
memory usage: 2.0+ MB
None


### Вывод

В данных были выявлены и успешно устранены пропуски и некореектные значения, по принципам указанных в комментариях к коду

### Замена типа данных <a id="4"></a>

In [30]:
# При просмотре таблицы не было выявленно несоответствующих типов данных данным, но были столбцы, где 'float64' можно было
# Заменить на 'int', что и было произведено

data['total_income'] = data['total_income'].astype('int')
data['days_employed'] = data['days_employed'].astype('int')
data['dob_years'] = data['dob_years'].astype('int')

#print(data.head())

print(data.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB
None


### Вывод

При просмотре таблицы не было выявленно несоответствующих типов данных данным, но были столбцы, где 'float64' можно было
заменить на 'int', что и было произведено

### Обработка дубликатов <a id="5"></a>

In [31]:
# При просмотре столбца 'education' методом '.value_counts() были выявлены дубликаты, возникновение которых 
# связано с разным регистром букв, что вызвано ошибкой человека вводившего данные
# В связи с этим было принято решение применить метод .'str_lower()'

data['education'] = data['education'].str.lower()
data['education'].value_counts()
print(data.duplicated().sum())

# Также было произведена проверка таблицы на полные дубликаты строк и дальнейшее их уничтожение.

data = data.drop_duplicates().reset_index(drop = True)
data.duplicated().sum()


71


0

### Вывод

Были вявлены и удалены дубликаты связанные с разным регистром, а также полные дубликаты строк

### Лемматизация <a id="6"></a>

In [32]:
# При просмотре таблицы в столбце 'purpose', было обнаружено, что цели кредита можно объеденить в более общие категории, 
# Тем самым облегчив их дальнейшую категоризацию
# Была произведена лемматизация столбца 'purpose', для определения основных категорий целей кредита

from pymystem3 import Mystem
m = Mystem()
uniqueVals = data['purpose'].unique()
myString = ','.join(uniqueVals)
lemmas = m.lemmatize(myString)
from collections import Counter
print(Counter(lemmas))

Counter({' ': 59, ',': 37, 'покупка': 10, 'недвижимость': 10, 'автомобиль': 9, 'образование': 9, 'жилье': 7, 'с': 5, 'операция': 4, 'на': 4, 'свой': 4, 'свадьба': 3, 'строительство': 3, 'получение': 3, 'высокий': 3, 'дополнительный': 2, 'для': 2, 'коммерческий': 2, 'жилой': 2, 'подержать': 2, 'заниматься': 2, 'сделка': 2, 'приобретение': 1, 'сыграть': 1, 'проведение': 1, 'семья': 1, 'собственный': 1, 'со': 1, 'профильный': 1, 'сдача': 1, 'ремонт': 1, '\n': 1})


### Категоризация данных <a id="7"></a>

In [33]:
# Для привязки строк к освноым целям кредита была написана функция 
# сравнивающая и возвращающая основную лемму столбца 'purpose' в столбец 'general_purpose'

def lemmarow(row):
    word = row['purpose']
    ret = ' '.join(m.lemmatize(word))
    if 'недвижимость' in ret:
        return 'недвижимость'
    if 'жилье' in ret:
        return 'недвижимость'
    if 'автомобиль' in ret:
        return 'автомобиль'
    if 'образование' in ret:
        return 'образование'
    if 'свадьба' in ret:
        return 'свадьба'
data['general_purpose'] = data.apply(lemmarow, axis = 1)
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,general_purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба


### Вывод

С помощью описанных в комментариях к коду действий, был создан столбец 'general_purpose' с основными категориями целей кредита

### Шаг 3. Ответы на вопросы <a id="8"></a>

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [34]:

#сгрупперуем таблицу по стобцам 'children' и 'debt'
data.groupby('children')['debt'].value_counts()


children  debt
0         0       13028
          1        1063
1         0        4410
          1         445
2         0        1926
          1         202
3         0         303
          1          27
4         0          37
          1           4
5         0           9
Name: debt, dtype: int64

In [35]:
print('Процент людей без детей имеющих кредитную задолженность = {:.1%}'.format(1063/13028))
print('Процент людей с 1 ребенком имеющих кредитную задолженность = {:.1%}'.format(445/4410))
print('Процент людей с 2 детьми имеющих кредитную задолженность = {:.1%}'.format(202/1926))
print('Процент людей с 3 детьми имеющих кредитную задолженность = {:.1%}'.format(27/303))
print('Процент людей с 4 детьми имеющих кредитную задолженность = {:.1%}'.format(4/37))
print('Люди с 5 детьми в данной выборке не имеют задолженностей по кредитам')



Процент людей без детей имеющих кредитную задолженность = 8.2%
Процент людей с 1 ребенком имеющих кредитную задолженность = 10.1%
Процент людей с 2 детьми имеющих кредитную задолженность = 10.5%
Процент людей с 3 детьми имеющих кредитную задолженность = 8.9%
Процент людей с 4 детьми имеющих кредитную задолженность = 10.8%
Люди с 5 детьми в данной выборке не имеют задолженностей по кредитам


### Вывод

Процент людей имеющих задолженность с 1, 2 и 4 детьми ≈ 10.4%
В то время как люди без детей и люди с 3 детьми имеют меньшее отношение по задолженности ≈ 8.5%

Значит ли это, что бог любит троицу? 
Никакой зависимости от количества детей к возврату кредита не наблюдается. Меньше всего задолженностей имеют люди без детей.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [36]:
# Сгруппируем таблицу по стобцам 'family_status' и 'debt

print(data.groupby('family_status')['debt'].value_counts())

family_status          debt
Не женат / не замужем  0        2536
                       1         274
в разводе              0        1110
                       1          85
вдовец / вдова         0         896
                       1          63
гражданский брак       0        3763
                       1         388
женат / замужем        0       11408
                       1         931
Name: debt, dtype: int64


In [37]:
print('Процент людей с семейным положением "Не женат / не замужем" имеющих кредитную задолженность = {:.1%}'.format(274/2536))
print('Процент людей с семейным положением "в разводе" имеющих кредитную задолженность = {:.1%}'.format(85/1110))
print('Процент людей с семейным положением "вдовец / вдова" имеющих кредитную задолженность = {:.1%}'.format(63/896))
print('Процент людей с семейным положением "гражданский брак" имеющих кредитную задолженность = {:.1%}'.format(388/3763))
print('Процент людей с семейным положением "женат / замужем" имеющих кредитную задолженность = {:.1%}'.format(931/11408))

Процент людей с семейным положением "Не женат / не замужем" имеющих кредитную задолженность = 10.8%
Процент людей с семейным положением "в разводе" имеющих кредитную задолженность = 7.7%
Процент людей с семейным положением "вдовец / вдова" имеющих кредитную задолженность = 7.0%
Процент людей с семейным положением "гражданский брак" имеющих кредитную задолженность = 10.3%
Процент людей с семейным положением "женат / замужем" имеющих кредитную задолженность = 8.2%


### Вывод

На основе произведенных расчетов можно заметить, что люди с семейным положением "Не женат / не замужем" и "гражданский брак" имеют практически одинаковое соотношение задолженности по возврату кредитов ≈ 10.5%. 

В то время как люди с семейным положением "вдовец / вдова" , "женат / замужем" и "в разводе" имеют соотношение ≈ 7.6%

Исходя из данных расчетов можно предположить, что люди находящиеся в браке, либо там побывавшие, ответственнее подходят к планированию собственного бюджета и рассчету кредита

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [38]:
#Для ответа на данный вопрос напишем функцию возврающую класс дохода в столбец 'income_class'

#print(data_grouped['total_income'].min())
#print(data_grouped['total_income'].median())
#print(data_grouped['total_income'].max())

def income_group(row):
    income = row['total_income']
    if income < 30000:
            return 'низкий'
    if income <= 100000:
            return 'средний'
    if income <= 300000:
            return 'высокий'
    return 'очень высокий'
data['income_class'] = data.apply(income_group, axis = 1)
#print(data.head(10))


In [39]:
# Сгруппируем таблицу по стобцам 'income_class' и 'debt

print(data.groupby('income_class')['debt'].value_counts())

income_class   debt
высокий        0       14228
               1        1281
низкий         0          20
               1           2
очень высокий  0        1376
               1         106
средний        0        4089
               1         352
Name: debt, dtype: int64


In [40]:
print('Процент людей с низким уровнем дохода и кредитной задолженностью = {:.1%}'.format(2/20))
print('Процент людей с средним уровнем дохода и кредитной задолженностью = {:.1%}'.format(352/4089))
print('Процент людей с высоким уровнем дохода и кредитной задолженностью = {:.1%}'.format(1281/14228))
print('Процент людей с очень высоким уровнем дохода и кредитной задолженностью = {:.1%}'.format(106/1376))

Процент людей с низким уровнем дохода и кредитной задолженностью = 10.0%
Процент людей с средним уровнем дохода и кредитной задолженностью = 8.6%
Процент людей с высоким уровнем дохода и кредитной задолженностью = 9.0%
Процент людей с очень высоким уровнем дохода и кредитной задолженностью = 7.7%


### Вывод

На основе полученной информации заметно, что соотношение задолженности по возврату кредитов постепенно снижается с ростом уровня дохода

Оно и понятно, чем больше денег, тем сложнее попасть в кредитную воронку

- Как разные цели кредита влияют на его возврат в срок?

In [41]:
print(data.groupby('general_purpose')['debt'].value_counts())

general_purpose  debt
автомобиль       0        3903
                 1         403
недвижимость     0       10029
                 1         782
образование      0        3643
                 1         370
свадьба          0        2138
                 1         186
Name: debt, dtype: int64


In [42]:
print('Процент людей взявших кредит на автомобиль имеющих крдитную задолженность = {:.1%}'.format(403/3903))
print('Процент людей взявших кредит на недвижимость имеющих крдитную задолженность = {:.1%}'.format(782/10029))
print('Процент людей взявших кредит на образование имеющих крдитную задолженность = {:.1%}'.format(370/3643))
print('Процент людей взявших кредит на свадьбу имеющих крдитную задолженность = {:.1%}'.format(186/2138))

Процент людей взявших кредит на автомобиль имеющих крдитную задолженность = 10.3%
Процент людей взявших кредит на недвижимость имеющих крдитную задолженность = 7.8%
Процент людей взявших кредит на образование имеющих крдитную задолженность = 10.2%
Процент людей взявших кредит на свадьбу имеющих крдитную задолженность = 8.7%


### Вывод

Меньше всего задолженностей имеют люди взявшие кредит на недвижимость и свадьбу, а больше всего на автомобиль и образование


### Шаг 4. Общий вывод <a id="9"></a>

Нужно было разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок.

Никакой зависимости от количества детей к возврату кредита не наблюдается. Меньше всего задолженностей имеют люди без детей и люди с 3 детьми.

Также люди с семейным положением "Не женат / не замужем" и "гражданский брак" имеют большее соотношение задолженности по возврату кредитов, чем люди находящиеся в браке, либо там побывавшие